# Baseline experiment 

The output of the first assessment of a pair is used as predicton for the second assessment.

In [ ]:
from constants import targets, get_feature_sets
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

X = pd.read_pickle('data/cached_X_imputed.pckl')
y = pd.read_pickle('data/cached_y.pckl')
feature_sets = get_feature_sets(X)
df = pd.read_pickle('data/cached_df.pckl')

ys_train, ys_test = [], []
lastrow = None

patient_ids = df.BALANCE_ID.unique()
for patient_id in patient_ids:
    pdf = df[df.BALANCE_ID == patient_id]
    for i in range(0, pdf.shape[0] - 1):
        for j in range(i+1, pdf.shape[0]):
            assessment0 = pdf.iloc[i].copy()
            assessment1 = pdf.iloc[j]
            
            time_diff = assessment1['Assessment_date_days'] - assessment0['Assessment_date_days']
            assessment0['time_diff'] = time_diff;
            
            # THIS IS THE ONLY CHANGE IN THIS NOTEBOOK COMPARED TO COMMON PREPROCESSING.
            ys_train.append(assessment0[['BALANCE_ID'] + targets])
            ys_test.append(assessment1[['BALANCE_ID'] + targets])           

In [ ]:
# Convert to DataFrames
y_train = pd.DataFrame(ys_train).reset_index(drop=True)
y_test = pd.DataFrame(ys_test).reset_index(drop=True)

### This baseline assumes that T0==T1 

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import resample
import numpy as np

results_LOCF = {}
n_bootstraps = 1000
rng = np.random.RandomState(42)  # For reproducibility

for target in targets:
    # Extract true labels and predicted probabilities for the target
    true_labels = y_test[target].astype(int)
    predicted_scores = y_train[target].astype(float)  # LOCF assumption

    # Calculate the base AUC
    auc_value = roc_auc_score(true_labels, predicted_scores)

    # Bootstrap AUCs
    bootstrapped_scores = []
    for i in range(n_bootstraps):
        # Bootstrap sample indices
        indices = rng.randint(0, len(predicted_scores), len(predicted_scores))
        if len(np.unique(true_labels.iloc[indices])) < 2:
            continue  # Skip if only one class in sample
        auc = roc_auc_score(true_labels.iloc[indices], predicted_scores.iloc[indices])
        bootstrapped_scores.append(auc)

    # Compute 95% confidence interval
    ci_lower, ci_upper = np.percentile(bootstrapped_scores, [2.5, 97.5])

    # Store results
    results_LOCF[target] = {
        "Classifier": "LOCF",
        "roc_auc": {
            "mean": auc_value,
            "95% CI": (ci_lower, ci_upper)
        }
    }

    print(f"{target} - ROC AUC: {auc_value:.3f} (95% CI: {ci_lower:.3f} - {ci_upper:.3f})")

In [ ]:
rows = []

for target, metrics in results_LOCF.items():
    row = {
        'Target': target,
        'Classifier': metrics['Classifier']
    }

    if 'roc_auc' in metrics:
        roc_auc = metrics['roc_auc']
        row['roc_auc'] = round(roc_auc['mean'], 3)
        row['roc_auc_ci_low'] = round(roc_auc['95% CI'][0], 3)
        row['roc_auc_ci_high'] = round(roc_auc['95% CI'][1], 3)

    rows.append(row)

# Create DataFrame
results_df = pd.DataFrame(rows)

# Export to CSV
results_df.to_csv('results_LOCF_AUC.csv', index=False)

In [ ]:
assert X.shape[0] == y_train.shape[0] == y_test.shape[0]